In [1]:
import polars as pl
import pandas as pd
from functions import stringVaziaToNull, stringCleanExtraSpaces, stringLowerCase
import json 
import time
from connDb import path_db

In [2]:
def getRedeSocialStg() -> pl.LazyFrame:
    """ Lendo os dados da tabela  stg_deputados e  filtrando as redes  sociais  
        nulas (5 é  um valor arbitrário, com 2 já é possível filtrar os nulos)
    """
    df = (
            pl.read_database_uri('SELECT "id", "redeSocial" from stg_deputados', path_db)
            .filter(pl.col("redeSocial")
            .str.len_bytes() > 5)
            .lazy()
    )

    return df

In [3]:

def transformRedeSocial(df: pl.LazyFrame) -> pl.DataFrame:
    """ Normalizando a coluna de Redes Sociais removendo caracteres especiais e divid
        -indo as urls por ','. Depois adicionando um contador que é utilizado em   se
        -guida para repetir o Id em uma lista correspondente às urls. Depois  explodi
        -mos as duas listas, resultando em uma tabela de mapeamento, onde cada url de
        rede social corresponde a um Id de deputado.
    """
    ldf = (
            df.with_columns(
                pl.col("redeSocial")
                .str.replace_all(r'[\[\]"]', '')
                .str.split(", ")
            )
            .with_columns(
                count=pl.col("redeSocial")
                .list.len()
            )
            .select(
                pl.col("id").repeat_by(pl.col("count")).alias("id_deputado"),
                pl.col("redeSocial")
            )
            .explode("id_deputado", "redeSocial")
            .with_columns(
                pl.col("id_deputado").cast(pl.Int64)
            )
    )
    
    ldf = stringVaziaToNull(ldf)
    
    ldf = stringCleanExtraSpaces(ldf)
    
    ldf = stringLowerCase(ldf)
    
    return ldf.collect()


In [4]:
def loadRedeSocial(df:pl.DataFrame) -> None:
    """ Carregando os dados tratados na tabela final 'rede_social',
        apta para ser utilizada em análises futuras. 
    """
    try:
        df.write_database("rede_social", path_db, if_exists="replace", engine="sqlalchemy")
    
    except:
        print("Erro ao carregar dados na tabela 'rede_social'\n")
        time.sleep(1)
    
    else:
        print("Sucesso ao carregar dados na tabela 'rede_social'\n")

In [5]:
def getDeputadosStg() -> pd.DataFrame:
    """ Lendo os dados da tabela  stg_deputados, que será utilizada para norma- 
        lização da coluna de 'ultimoStatus', que em sua origem se encontra no
        formato de string json
    """
    df = pd.read_sql_query('SELECT * from stg_deputados', path_db)

    return df



In [6]:
def transformDeputados(df:pd.DataFrame) -> pl.DataFrame:
    """ Realizando as etapas de limpeza e transformação da tabela de deputados.
        A princípio estaremos normalizando a coluna 'ultimoStatus', passando-a
        do formato string json para novos conjuntos de colunas. Nesta  etapa a
        solução mais prática foi adquirida através do Pandas, poréma as etapas
        seguintes (de limpeza e padronização das demais strings) foram realiza-
        das utilizando expressões do Polars  (uma biblioteca mais perfomática)
    """
    df['ultimoStatus'] = df['ultimoStatus'].apply(json.loads)
    
    df = pd.concat([df.drop(['ultimoStatus'], axis=1), df['ultimoStatus'].apply(pd.Series)], axis=1).drop(columns=['gabinete'])

    ldf = pl.LazyFrame(df)
    
    ldf = stringVaziaToNull(ldf)
    
    ldf = stringCleanExtraSpaces(ldf)
    
    ldf = stringLowerCase(ldf)

    return ldf.collect()

In [7]:
def loadDeputados(df:pl.DataFrame) -> None:
    """ Carregando os dados tratados na tabela final 'deputados', 
        apta para ser utilizada em análises futuras. 
    """
    try:
        df.write_database("deputados", path_db, if_exists="replace", engine="sqlalchemy")
    
    except:
        print("Erro ao carregar dados na tabela 'deputados'\n")
        time.sleep(1)
    
    else:
        print("Sucesso ao carregar dados na tabela 'deputados'\n")



In [8]:
df = getRedeSocialStg()
df = transformRedeSocial(df)
loadRedeSocial(df)


Sucesso ao carregar dados na tabela 'rede_social'

